In [2]:
%load_ext autoreload
%autoreload 2 

In [3]:
import numpy as np
import pandas as pd

In [4]:
v13_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v13.csv')[['id', 'signal_depth', 'target']]
v14_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v14.csv')[['id', 'signal_depth', 'target']]
v15_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v15.csv')[['id', 'signal_depth', 'target']]
# v16_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v16.csv')[['id', 'signal_depth', 'target']]
# v17_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v17.csv')[['id', 'signal_depth', 'target']]
# train = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/train_labels.csv')
v18n1 = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v18n1.csv')
v18n2 = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v18n2.csv')
valid = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v18v.csv')
test = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/sample_submission.csv')
# train = train.loc[train['target'] != -1]
# v2_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v2.csv')

In [5]:
v13_df['path'] = v13_df['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v13/{x}.pickle')
v14_df['path'] = v14_df['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v14/{x}.pickle')
v15_df['path'] = v15_df['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v15/{x}.pickle')
valid['path'] = valid['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v18v/{x}.pickle')
v18n1['path'] = v18n1['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v18n1/{x}.pickle')
v18n2['path'] = v18n2['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v18n2/{x}.pickle')

In [6]:
# v13_df['group'] = v13_df['id'].apply(lambda x: x.split('_')[0])
# v14_df['group'] = v14_df['id'].apply(lambda x: x.split('_')[0])
# v15_df['group'] = v15_df['id'].apply(lambda x: x.split('_')[0])

In [7]:
v13_df['id'] = v13_df['id'] + '_v13'
v14_df['id'] = v14_df['id'] + '_v14'
v15_df['id'] = v15_df['id'] + '_v15'
v18n1['id'] = v18n1['id'] + '_v18n1'
v18n2['id'] = v18n2['id'] + '_v18n2'

In [12]:
# valid.to_csv('input/g2net-detecting-continuous-gravitational-waves/v18v.csv', index=False)

In [23]:
# # pd.concat([v0_df, v1_df, v2_df]).reset_index(drop=True).to_csv(
# pd.concat([v1_df, v4_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v1_v4.csv', index=False)

# pd.concat([v1_df, v4_df, v5_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v1_v4_v5.csv', index=False)

# pd.concat([v1_df, v4_df, v5_df, v6_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v1_v4_v5_v6.csv', index=False)

# pd.concat([v13_df, v14_df, v15_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v13_v14_v15.csv', index=False)

# pd.concat([v13_df, v14_df, v15_df, v16_df, v17_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v13_v14_v15_v16_v17.csv', index=False)

# pd.concat([v16_df, v17_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v16_v17.csv', index=False)

pd.concat([v18n1, v18n2]).reset_index(drop=True).to_csv(
    'input/g2net-detecting-continuous-gravitational-waves/concat_v18n1_v18n2.csv', index=False)

In [6]:
test['path'] = test['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/test/{x}.pickle')
test

,id,target,path
0,00054c878,0.5,input/g2net-detecting-continuous-gravitational...
1,0007285a3,0.5,input/g2net-detecting-continuous-gravitational...
2,00076c5a6,0.5,input/g2net-detecting-continuous-gravitational...
3,001349290,0.5,input/g2net-detecting-continuous-gravitational...
4,001a52e92,0.5,input/g2net-detecting-continuous-gravitational...
...,...,...,...
7970,ffbce04ef,0.5,input/g2net-detecting-continuous-gravitational...
7971,ffc2d976b,0.5,input/g2net-detecting-continuous-gravitational...
7972,ffc905909,0.5,input/g2net-detecting-continuous-gravitational...
7973,ffe276f3e,0.5,input/g2net-detecting-continuous-gravitational...


In [9]:
test.to_csv('input/g2net-detecting-continuous-gravitational-waves/test.csv', index=False)

# Add metadata to valid csv

In [56]:
import pickle
from prep_data_v18v import to_spectrogram
from transforms import adaptive_resize
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [18]:
valid = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v18v.csv')

In [60]:
# valid_freq = []
# valid_nonstaionary = []
# valid_artifact = []
# for i, gid in enumerate(tqdm(valid['id'].values)):
#     with open(f'input/g2net-detecting-continuous-gravitational-waves/v18v/{gid}.pickle', 'rb') as f:
#         data = pickle.load(f)[gid]
#     freq = data['frequency_Hz'].mean()
#     spec_h1 = adaptive_resize(to_spectrogram(data['H1']['SFTs']*1e22)[:, :, None], 360, np.mean)
#     ts_h1 = data['H1']['timestamps_GPS']
#     spec_l1 = adaptive_resize(to_spectrogram(data['L1']['SFTs']*1e22)[:, :, None], 360, np.mean)
#     ts_l1 = data['L1']['timestamps_GPS']
#     asd_h1 = spec_h1.mean(axis=(1, 2)).reshape(180, 2).mean(1)
#     std_h1 = spec_h1.std()
#     min_h1 = spec_h1.min()
#     peak_h1 = np.max((spec_h1 - min_h1) / std_h1)
#     asd_l1 = spec_l1.mean(axis=(1, 2)).reshape(180, 2).mean(1)
#     std_l1 = spec_l1.std()
#     min_l1 = spec_l1.min()
#     peak_l1 = np.max((spec_l1 - min_l1) / std_h1)
#     valid_freq.append(freq)
#     valid_nonstaionary.append(int(np.percentile(asd_h1, 90)-np.percentile(asd_h1, 10) / asd_h1.mean() > 1.5))
#     valid_artifact.append(int(max(peak_h1, peak_l1) > 25))
#     # print(gid, freq, np.percentile(asd_h1, 90)-np.percentile(asd_h1, 10) / asd_h1.mean(), peak_h1)
#     # if np.percentile(asd_h1, 90)-np.percentile(asd_h1, 10) / asd_h1.mean() >  1.5:
#     #     plt.imshow(spec_h1)
#     #     plt.show()
#     #     plt.close()
#     # if i == 50:
#     #     break
# valid['freq'] = valid_freq
# valid['nonstationary'] = valid_nonstaionary
# valid['artifact'] = valid_artifact
# valid.to_csv('input/g2net-detecting-continuous-gravitational-waves/v18v.csv', index=False)